In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION=os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_DEPLOYMENT=os.getenv("AZURE_OPENAI_DEPLOYMENT")

In [ ]:
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Use the current user identity to authenticate with Azure OpenAI, Cognitive Search and Blob Storage (no secrets needed, 
# just use 'az login' locally, and managed identity when deployed on Azure). If you need to use keys, use separate AzureKeyCredential instances with the 
# keys for each service
# If you encounter an error here are some ways to troubleshoot:
# - you can exclude the problematic credential by using a parameter (ex. exclude_shared_token_cache_credential=True)
# - For your credential, grant Cognitive Services User and Cognitive Services OpenAI User roles - EVEN IF YOU HAVE OWNER/ADMIN RIGHTS.
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

client = AzureOpenAI(
  azure_endpoint = AZURE_OPENAI_ENDPOINT,
  api_version=AZURE_OPENAI_API_VERSION,
  azure_ad_token_provider=token_provider
)

In [ ]:
IMAGE_PATH="../media/image-whiteboard-stickynotes.png"

In [ ]:
import base64

def executeGptVision(encoded_image, system_prompt_file=None, system_prompt=None):
    if system_prompt_file:
        with open(system_prompt_file, "r") as file:
            system_prompt = file.read()
    elif system_prompt is None:
        raise ValueError("Either filename or system_prompt must be provided.")

    messages = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encoded_image}"
                    }
                }
            ]
        }
    ]

    response = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,
        messages=messages,
        temperature=0
    )

    return response.choices[0].message.content

In [ ]:
encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')
noteType = executeGptVision(encoded_image, system_prompt_file="../src/prompts/detectNoteType.txt")
print (noteType)

In [ ]:
if noteType == "PAPER":
    ocrFile = "../src/prompts/ocrPaper.txt"
elif noteType == "WHITEBOARD":
    ocrFile = "../src/prompts/ocrWhiteboard.txt"
else:
    ocrFile = "../src/prompts/ocrImage.txt"

extractedText = executeGptVision(encoded_image, system_prompt_file=ocrFile)
print (extractedText)

In [ ]:
def executeCompletion(text, system_prompt_file=None, system_prompt=None):
    if system_prompt_file:
        with open(system_prompt_file, "r") as file:
            system_prompt = file.read()
    elif system_prompt is None:
        raise ValueError("Either filename or system_prompt must be provided.")

    messages = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": text
        }
    ]

    response = client.chat.completions.create(
        model="gpt4o",
        messages=messages,
        temperature=0.3
    )

    return response.choices[0].message.content

In [ ]:
if noteType == "PAPER" or noteType == "WHITEBOARD":
    extractedText = executeCompletion(text=extractedText, system_prompt_file="../src/prompts/proofread.txt")
    print (extractedText)

In [ ]:
if noteType == "PAPER" or noteType == "WHITEBOARD":
    extractedText = executeCompletion(extractedText, system_prompt_file="../src/prompts/sectionHeader.txt")
    print (extractedText)

In [ ]:
extractedText = extractedText.strip()
if extractedText.startswith("```markdown"):
    extractedText = extractedText[len("```markdown"):]
if extractedText.endswith("```"):
    extractedText = extractedText[:-len("```")]

extractedText = extractedText.strip()

In [ ]:
import os
import datetime

filename = os.path.basename(IMAGE_PATH)
last_updated = datetime.datetime.fromtimestamp(os.path.getmtime(IMAGE_PATH)).strftime('%Y-%m-%d %H:%M')

result = f"""---
note-type: {noteType}
created-date: {last_updated}
last-updated: {last_updated}
---
# {filename}
![{filename}]({IMAGE_PATH})
{extractedText}"""

# save result to output.md
with open("output.md", "w") as file:
    file.write(result)

print(result)
